In [1]:
import pandas as pd
from tqdm import tqdm
from pandarallel import pandarallel
# import pyranges as pr

# We try to match them

In [ ]:
df_gencode_introns = pd.read_csv("gencode_v44_introns_modded")
df_gencode_introns

In [ ]:
df_recount3_test = df_recount3.copy()
df_recount3_test

In [ ]:
df_recount3_test.set_index(["Strand", "Start", "End"], inplace=True)

In [ ]:
df_recount3_test.sort_index(inplace=True)
df_recount3_test

In [ ]:
def match_recount(row):
    # print(row)
    # print(row.chrom)
    # try:
        # score = df_recount3_test.loc[(row.strand, row.intron_start, row.intron_ends), ["Features", "Score"]].values
    # !echo 'tabix ftp-data/recount3.bed.gz {row.chrom}:{row.intron_start}-{row.intron_ends}'
    matched_rows = !tabix ftp-data/recount3.bed.gz {row.chrom}:{row.intron_start}-{row.intron_ends} | \
        awk -v start={row.intron_start} -v stop={row.intron_ends} '$2 == start && $3 == stop'
    # score = !tabix ftp-data/recount3.bed.gz chr1:12227-12612 | awk -v start=12227 -v stop=12612 '$2 == start && $3 == stop'

        #"Chrom", "Score", "GTExv2", "TCGAv2", "SRAv3h", "Splice_site"
    # except:
    #     score = [None, -1]
    #     score = "Unmatched"
    # print(df_matched)
    # print(matched_rows)
    # print(type(score))
    return matched_rows

In [ ]:
pandarallel.initialize(progress_bar=True)

In [ ]:
df_gencode_introns["RC3_matches"] = df_gencode_introns.parallel_apply(match_recount, axis=1)
df_gencode_introns

In [ ]:
df_gencode_introns.to_csv("gencode_v44_introns_with_recount3_score.csv", index=False)

In [ ]:
!tabix ftp-data/recount3.bed.gz chr1:12227-12612 | awk -v start=12227 -v stop=12612 '$2 == start && $3 == stop'

In [ ]:
df_test = df_gencode_introns.copy().head(500)
df_test["RC3"] = df_test.apply(match_recount, axis=1)
df_test

In [ ]:
!tabix ftp-data/recount3.bed.gz chr1:12697-12974 | head -n 1

In [ ]:
!tabix ftp-data/recount3.bed.gz chr1:12057-12178

In [ ]:
df_recount3 = pd.read_csv("ftp-data/recount3.bed", sep="\t",
                          names=["Chrom", "Start", "End", "Features", "Score", "Strand"])
df_gencode_introns = pd.read_csv("gencode_v44_introns.csv")
df_recount3
df_featuers_split = df_recount3["Features"].str.split(";")
df_featuers_split
df_recount3["GTExv2"] = df_featuers_split.str[0].str.split("=").str[1]
df_recount3["TCGAv2"] = df_featuers_split.str[1].str.split("=").str[1]
df_recount3["SRAv3h"] = df_featuers_split.str[2].str.split("=").str[1]
df_recount3["Splice_site"] = df_featuers_split.str[3]
df_recount3


def split_features(row):
    # Split the "Features" column by semicolon
    # print("OK")
    # print(row)
    features_split = row.split(";")
    # print(features_split)
    # Extract the desired values from the split columns and assign them to new columns
    GTExv2 = features_split[0].split("=")[1]
    TCGAv2 = features_split[1].split("=")[1]
    SRAv3h = features_split[2].split("=")[1]
    Splice_site = features_split[3]
    return pd.Series([GTExv2, TCGAv2, SRAv3h, Splice_site])

In [ ]:
df_recount3 = pd.read_csv("ftp-data/recount3.bed", sep="\t",
                          names=["Chrom", "Start", "End", "Features", "Score", "Strand"])

In [ ]:
df_recount3.set_index(["Strand", "Start", "End"], inplace=True)

In [ ]:
df_recount3.sort_index(inplace=True)

In [ ]:
df_recount3

In [ ]:
chrom_uniques = list(df_recount3.Chrom.unique())
df_chrom = {}

In [ ]:
# Splitting the dataframe rows separately
for chrom in tqdm(chrom_uniques):
    df_chrom[chrom] = df_recount3[df_recount3.Chrom == chrom]

df_chrom

In [ ]:
df_chrom["chr1"]

In [ ]:
# df_chr1 = df_chrom["chr1"]
def match_recount(row):
    # print(row)
    # print(row.chrom)
    try:
        score = df_chrom[row.chrom].loc[(row.strand, row.intron_start, row.intron_ends), ["Features", "Score"]].values
    # !echo 'tabix ftp-data/recount3.bed.gz {row.chrom}:{row.intron_start}-{row.intron_ends}'

    # score = !tabix ftp-data/recount3.bed.gz chr1:12227-12612 | awk -v start=12227 -v stop=12612 '$2 == start && $3 == stop'

        #"Chrom", "Score", "GTExv2", "TCGAv2", "SRAv3h", "Splice_site"
    except:
        score = [None, -1]
    #     score = "Unmatched"
    # print(df_matched)
    # print(matched_rows)
    # print(type(score))
    # print(score)
    return score

In [ ]:
pandarallel.initialize(progress_bar=True)

In [ ]:
df_test = df_gencode_introns.copy().head(50)
df_test["RC3"] = df_test.apply(match_recount, args=(), axis=1)
df_test

In [ ]:
df_gencode_introns = pd.read_csv("gencode_v44_introns.csv")


In [ ]:
df_chr1_gencode = df_gencode_introns[df_gencode_introns.chrom == "chr1"]
df_chr1_gencode

In [ ]:
df_test = df_chr1_gencode.copy().head(50)

df_test.apply(match_recount, axis=1)

In [ ]:
df_chr1.progress_apply(match_recount, axis=1)